<a href="https://colab.research.google.com/github/leedonghae12/test2/blob/main/%EB%94%A5%EB%9F%AC%EB%8B%9D%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

LDA 사용

In [1]:
!pip install gensim
!pip install nltk
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 49.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 36.6 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import gensim
from gensim import corpora
from gensim.models.ldamodel import LdaModel
from konlpy.tag import Mecab
from nltk.corpus import stopwords
import re

In [3]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

삼양식품 뉴스 데이터 갖고오기

In [4]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/leedonghae12/test2/main/samyang.csv")

In [5]:
df = df.loc[df['title'].str.contains("삼양")]

In [6]:
df.shape

(942, 6)

In [7]:
# Convert date column to datetime
df['date'] = pd.to_datetime(df['date'])
# Extract year and month
df['year_month'] = df['date'].dt.to_period('M')

In [8]:
df.tail()

,ID,date,title,summary,press,URL,year_month
3141,1.100501e+06,1970-01-01 00:00:00.020230102,"김윤 삼양 회장, “위기 극복하고 당당하게 100주년 준비”",올해로 창립 99주년을 맞는 삼양그룹이 어려운 경영 환경을 슬기롭게 극복해 새로운 ...,문화일보,http://www.munhwa.com/news/view.html?no=202301...,1970-01
3142,4.101008e+06,1970-01-01 00:00:00.020230102,"[신년사]김윤 삼양그룹 회장 ""새로운 100년 초석 마련""",[이데일리 하지나 기자] 김윤 삼양그룹 회장은 2일 신년사를 통해 “올해 경영환경은...,이데일리,http://www.edaily.co.kr/news/newspath.asp?news...,1970-01
3143,1.101101e+06,1970-01-01 00:00:00.020230102,"'불닭신화' 삼양식품, 올해도 해외 공장 없이 ""수출 확대"" 그 자신감의 근거는",올 한 해 신종 코로나바이러스 감염증(코로나19) 여파로 가공식품 소비가 크게 늘었...,한국일보,https://hankookilbo.com/News/Read/A20230101164...,1970-01
3144,2.100201e+06,1970-01-01 00:00:00.020230102,"김윤 삼양 회장 ""미래 100년 향한 도전에 적극 동참해 달라""",[머니투데이 김성은 기자] \n삼양그룹이 창립 100주년을 한 해 앞두고 어려운 경...,머니투데이,http://news.moneytoday.co.kr/view/mtview.php?n...,1970-01
3145,2.100201e+06,1970-01-01 00:00:00.020230102,"김정수 삼양식품 부회장 ""글로벌 톱 100 기업 진입 노력해야""",[머니투데이 박미주 기자] 김정수 삼양식품 부회장이 글로벌 상위 100위권 기업으로...,머니투데이,http://news.moneytoday.co.kr/view/mtview.php?n...,1970-01


토큰화


In [9]:
from konlpy.tag import Komoran
from nltk.corpus import stopwords
import re

In [10]:
def preprocess_text(text):
    # 특수 문자 제거
    text = re.sub(r'[^ ㄱ-ㅣ가-힣]+', '', text)

    # 형태소 분석기 생성
    komoran = Komoran()

    # 형태소 단위로 분리
    tokens = komoran.morphs(text)

    return tokens

In [11]:
file_path = 'samyang.csv'

In [12]:
def load_news_data(file_path):
    df = pd.read_csv("https://raw.githubusercontent.com/leedonghae12/test2/main/samyang.csv")
    return df

In [23]:
!pip install pyLDAvis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 9.3 MB/s eta 0:00:00


In [24]:
from gensim import corpora
from gensim.models import LdaModel
import pyLDAvis.gensim_models
import pyLDAvis

In [13]:
df['tokens'] = df['title'].apply(preprocess_text)

In [20]:
dictionary = corpora.Dictionary(df['tokens'])
corpus = [dictionary.doc2bow(tokens) for tokens in df['tokens']]

전체 뉴스에 대해 토픽 정하기

In [21]:
# Train LDA model
num_topics = 6  # 12개월여의 데이터, 두달에 하나의 토픽 가정
lda = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=15)

In [25]:
# Prepare data for visualization
vis_data = pyLDAvis.gensim_models.prepare(lda, corpus, dictionary)

# Display visualization
pyLDAvis.display(vis_data)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


각 뉴스를 토픽과 매칭

In [26]:
def assign_topic_to_title(tokens):
    bow = dictionary.doc2bow(tokens)
    topic_distribution = lda[bow]
    main_topic = max(topic_distribution, key=lambda x: x[1])[0]
    return main_topic

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [27]:
df['topic'] = df['tokens'].apply(assign_topic_to_title)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [28]:
df.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,ID,date,title,summary,press,URL,year_month,tokens,topic
0,2.100801e+06,1970-01-01 00:00:00.020240531,불닭볶음면 열풍 국물라면으로 삼양식품 '맵탱',삼양식품은 불닭볶음면이 주도한 매운맛 열풍을 국물 라면으로 이어가기 위해 지난해 8...,아시아경제,https://view.asiae.co.kr/article/2024053109401...,1970-01,"[불닭볶음면, 열풍, 국물, 라면, 으로, 삼양식품, 맵탱]",2
15,2.100851e+06,1970-01-01 00:00:00.020240529,"하나證 ""삼양식품, 해외 매출 비중 확대로 주가 상승 여력 충분""",삼양식품 전경 [사진=삼양식품] \n하나증권은 29일 삼양식품에 대해 올해 수출 비...,아주경제,https://www.ajunews.com/view/20240529082109794,1970-01,"[하나, 삼양식품, 해외, 매출, 비중, 확대, 로, 주가, 상승, 여력, 충분]",1
16,2.100101e+06,1970-01-01 00:00:00.020240529,“올해 주가 두 배 넘게 올랐어도 더 간다”...실적대비 저평가라는 삼양식품,하나증권은 29일 삼양식품에 대해 글로벌 피어 대비 실적 모멘텀이 강하나 밸류에이션...,매일경제,http://www.mk.co.kr/article/11027368,1970-01,"[올해, 주가, 두, 배, 넘, 게, 오르, 았, 어도, 더 간, 다실, 적, 대비...",0
17,2.100201e+06,1970-01-01 00:00:00.020240529,'수출 대장주' 삼양식품 밸류에이션 상승 여력 여전하다-하나증권,[머니투데이 천현정 기자] 하나증권은 수출 대장주가 된 삼양식품의 밸류에이션(가치 ...,머니투데이,http://news.moneytoday.co.kr/view/mtview.php?n...,1970-01,"[수출, 대, 장주, 삼양식품, 배, ㄹ, 류, 에이, 션, 상승, 여력, 여전, ...",1
18,1.100751e+06,1970-01-01 00:00:00.020240529,"""삼양식품, 2분기 中 쇼핑제 등 포진 단기 실적 모멘텀 강해""",아시아투데이 김동민 기자 = 하나증권은 29일 삼양식품에 대해 2분기 중국 쇼핑제 ...,아시아투데이,https://www.asiatoday.co.kr/view.php?key=20240...,1970-01,"[삼양식품, 분기, 쇼핑, 제, 등, 포진, 단기, 실적, 모멘텀, 강하, 아]",4


월별로 토픽 메인 선정

In [29]:
def get_main_topic(group):
    topic_counts = group['topic'].value_counts()
    main_topic = topic_counts.idxmax()
    return main_topic

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [30]:
# Assuming 'date' is a column with datetime objects
df['year_month'] = df['date'].dt.to_period('M')

main_topics = df.groupby('year_month').apply(get_main_topic).reset_index()
main_topics.columns = ['year_month', 'main_topic']

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [31]:
main_topics = df.groupby('year_month').apply(get_main_topic).reset_index()
main_topics.columns = ['year_month', 'main_topic']

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [32]:
main_topics.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,year_month,main_topic
0,1970-01,1


각 월별 메인토픽에 해당하는 뉴스만 남기기

In [33]:
df = df.merge(main_topics, on = ['year_month'])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [34]:
df.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,ID,date,title,summary,press,URL,year_month,tokens,topic,main_topic
0,2.100801e+06,1970-01-01 00:00:00.020240531,불닭볶음면 열풍 국물라면으로 삼양식품 '맵탱',삼양식품은 불닭볶음면이 주도한 매운맛 열풍을 국물 라면으로 이어가기 위해 지난해 8...,아시아경제,https://view.asiae.co.kr/article/2024053109401...,1970-01,"[불닭볶음면, 열풍, 국물, 라면, 으로, 삼양식품, 맵탱]",2,1
1,2.100851e+06,1970-01-01 00:00:00.020240529,"하나證 ""삼양식품, 해외 매출 비중 확대로 주가 상승 여력 충분""",삼양식품 전경 [사진=삼양식품] \n하나증권은 29일 삼양식품에 대해 올해 수출 비...,아주경제,https://www.ajunews.com/view/20240529082109794,1970-01,"[하나, 삼양식품, 해외, 매출, 비중, 확대, 로, 주가, 상승, 여력, 충분]",1,1
2,2.100101e+06,1970-01-01 00:00:00.020240529,“올해 주가 두 배 넘게 올랐어도 더 간다”...실적대비 저평가라는 삼양식품,하나증권은 29일 삼양식품에 대해 글로벌 피어 대비 실적 모멘텀이 강하나 밸류에이션...,매일경제,http://www.mk.co.kr/article/11027368,1970-01,"[올해, 주가, 두, 배, 넘, 게, 오르, 았, 어도, 더 간, 다실, 적, 대비...",0,1
3,2.100201e+06,1970-01-01 00:00:00.020240529,'수출 대장주' 삼양식품 밸류에이션 상승 여력 여전하다-하나증권,[머니투데이 천현정 기자] 하나증권은 수출 대장주가 된 삼양식품의 밸류에이션(가치 ...,머니투데이,http://news.moneytoday.co.kr/view/mtview.php?n...,1970-01,"[수출, 대, 장주, 삼양식품, 배, ㄹ, 류, 에이, 션, 상승, 여력, 여전, ...",1,1
4,1.100751e+06,1970-01-01 00:00:00.020240529,"""삼양식품, 2분기 中 쇼핑제 등 포진 단기 실적 모멘텀 강해""",아시아투데이 김동민 기자 = 하나증권은 29일 삼양식품에 대해 2분기 중국 쇼핑제 ...,아시아투데이,https://www.asiatoday.co.kr/view.php?key=20240...,1970-01,"[삼양식품, 분기, 쇼핑, 제, 등, 포진, 단기, 실적, 모멘텀, 강하, 아]",4,1


In [35]:
df1 = df.loc[df['topic'] == df['main_topic']]
df1.shape #820

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(196, 10)

각 뉴스의 긍부정 파악

In [36]:
from transformers import AutoTokenizer, TextClassificationPipeline, AutoModelForSequenceClassification

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [37]:
tokenizer = AutoTokenizer.from_pretrained('kwoncho/KoFinBERT')
model     = AutoModelForSequenceClassification.from_pretrained('kwoncho/KoFinBERT')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/628 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/752k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/979 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/443M [00:00<?, ?B/s]

In [38]:
newslist = df1['title'].to_list()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [39]:
text_classifier = TextClassificationPipeline(
            tokenizer = tokenizer,
            model     = model,
            top_k     = None
        )

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [40]:
rlt = text_classifier(newslist)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [41]:
label = [r[0]['label'] for r in rlt]
label[:5]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['positive', 'positive', 'positive', 'neutral', 'neutral']

In [42]:
df1['label'] = label

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-42-50a42fa3d605>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['label'] = label


월별로 긍부정 카운트

In [43]:
grouped = df1.groupby(['year_month', 'label']).size().reset_index(name='count')
grouped.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,year_month,label,count
0,1970-01,negative,9
1,1970-01,neutral,100
2,1970-01,positive,87


In [44]:
pivot_table = grouped.pivot(index='year_month', columns='label', values='count').fillna(0)
pivot_table = pivot_table.reset_index()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [45]:
pivot_table.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


label,year_month,negative,neutral,positive
0,1970-01,9,100,87


In [46]:
pivot_table['p'] = (pivot_table['positive'] - pivot_table['negative'])/pivot_table['neutral']

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [47]:
pivot_table[['year_month', 'p']]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


label,year_month,p
0,1970-01,0.78
